<a href="https://colab.research.google.com/github/DietzscheNostoevsky/Learning_Pytorch/blob/main/02_Pytorch_NN_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification using Neural Networks in PyTorch

# 1. Data Generation

In [3]:
from sklearn.datasets import make_circles 

# Making 1000 samples 
n_samples = 1000

# creating circles 

X, y = make_circles(n_samples, 
                    noise=0.03, # addded noise 
                    random_state=42) # for replication 
                
print(F"X_values (Features) : \n{X[:5]}\n")
print(F"y_values (labels) : \n{y[:5]}")

X_values (Features) : 
[[ 0.75424625  0.23148074]
 [-0.75615888  0.15325888]
 [-0.81539193  0.17328203]
 [-0.39373073  0.69288277]
 [ 0.44220765 -0.89672343]]

y_values (labels) : 
[1 1 1 1 0]
